# Issues with lodash-fp

`lodash-fp` is nice but some functions do not work as well as they did in lodash itself.

## Answer from the maintainer

The maintainer has kindly responsed to the accompanying [bug report 1343](https://github.com/lodash/lodash/issues/1343):

> Having auto-curried iteratee-first data-last methods comes with some restrictions. This means **functions do not accept a variable number of arguments**. So methods that do are split out. For example there's `_.uniq` and `_.uniqBy`. We limit the number of arguments provided to `_.map` and friend's iteratee because it's easier for devs writing in that style to compose methods. Most don't want the index and object params messing up their composition.
> If you're wanting to provide the default value for `_.result` that would be a feature request on lodash-fp.

## Issues and limitations

In [2]:
var _ = require('lodash-fp');
var lodash = require('lodash');

undefined

### No way to provide the default value to `result`?!

In lodash, `_.result({}, "noSuchProperty", "default")` returns `"default"`.
In lodash-fp we put the object to the end but there is no (clear) way to provide the default param:

In [3]:
[_.result("default", "noSuchProperty", {}), 
 _.result("noSuchProperty", "default", {}), 
 _.result("noSuchProperty", {}, "default")]

[ undefined, undefined, undefined, undefined ]

### `assign` behaves strangely

The correct way to call `assign` in lodash-fp is with the object being changed in the last position:

In [5]:
_.assign({a:1}, {b:2}, {}); // => {a:1, b:2} as expected

{ b: 2, a: 1 }

However if we by a chance put an empty object to the first place, we get something weird:

In [7]:
_.assign({}, {a:1}, {b:2}); // expected: {a:1, b:2} (where the input b:2 object is modified)

{ a: 1 }

(Reported as [lodash-fp/issues/16](https://github.com/lodash/lodash-fp/issues/16).)

### zipObject(pairs) is broken

`zipObject([key, val, key2, val2, ..])` does not work anymore - it returns a fn expecting the vals array:

In [ ]:
_.zipObject(["key", "val"]); // expected => {key: "val"}

### Some (all?) functions processing objects get only the value, not the key param:

#### Find on objects doesn't get the key parameter

In [3]:
lodash.find({key:"val"}, console.log.bind(console)) // => value, key, the whole object

val key { key: 'val' }


undefined

In [5]:
_.find(console.log.bind(console), {key:"val"}) // => only val :-(

val


undefined

#### the same applies for map

In [8]:
_.map(_.partial(console.log.bind(console), "lodash-fp - key only: "), {a:1})
lodash.map({a:1}, _.partial(console.log.bind(console), "lodash - key, val, obj: "))

lodash-fp - key only:  1
lodash - key, val, obj:  1 a { a: 1 }


{ a: undefined }

#### and mapValues

In [9]:
_.mapValues(_.partial(console.log.bind(console), "lodash-fp - key only: "), {a:1})
lodash.mapValues({a:1}, _.partial(console.log.bind(console), "lodash - key, val, obj: "))

lodash-fp - key only:  1
lodash - key, val, obj:  1 a { a: 1 }


{ a: undefined }

### Max does not accept an iteratee argument:

In [10]:
_.max("n", [{n: 3},{n: 4}])

-Infinity

In [11]:
lodash.max([{n: 3},{n: 4}], "n")

{ n: 4 }